In [78]:
import sys
sys.path.append('../framework')
from NetworkClass import Network
import torch.nn.functional as F
from torch import nn

In [79]:
model = Network({
        "network":{
            'input_layer': {
                "units": 784,
                
                },
            'hidden_layer': [{
                    "units": 400, 
                    "type": "Linear"
                }, 
                {
                    "units": 50, 
                    "activation": "relu",
                    "type": "Linear"

                }],
            'output_layer': {
                "units": 10,
                "activation": "softmax",
                "type": "Linear"
                }
        }
    })

softmax


In [80]:
for (layer, param) in enumerate(model.parameters()):
    print("Layer {} , Parameters: {}".format(layer, param.shape))

Layer 0 , Parameters: torch.Size([400, 784])
Layer 1 , Parameters: torch.Size([400])
Layer 2 , Parameters: torch.Size([50, 400])
Layer 3 , Parameters: torch.Size([50])
Layer 4 , Parameters: torch.Size([10, 50])
Layer 5 , Parameters: torch.Size([10])


In [81]:
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision
import torch
import torch.nn as nn


In [82]:
batch_size_train = 64
batch_size_test = 1000
n_epochs = 3
learning_rate = 0.01
momentum = 0.5
log_interval = 10
random_seed = 1

In [83]:
class ReshapeTransform:
    def __init__(self, new_size):
        self.new_size = new_size

    def __call__(self, img):
        return torch.reshape(img, self.new_size)

In [84]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../data/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,)), ReshapeTransform((-1,))
                             ])),
  batch_size=batch_size_train, shuffle=True)


test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('../data/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,)), ReshapeTransform((-1,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [85]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [86]:
example_data.shape

torch.Size([1000, 784])

In [87]:
# import matplotlib.pyplot as plt

# fig = plt.figure()
# for i in range(6):
#   plt.subplot(2,3,i+1)
#   plt.tight_layout()
#   plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
#   plt.title("Ground Truth: {}".format(example_targets[i]))
#   plt.xticks([])
#   plt.yticks([])

In [88]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,
                      momentum=momentum)

In [89]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [90]:
def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

In [91]:
def test():
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = model(data)
      test_loss += criterion(output, target).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [181]:
test()

# Pruning every 3 epochs
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()
    



Test set: Avg. loss: 0.0020, Accuracy: 4168/10000 (42%)



RuntimeError: The size of tensor a (399) must match the size of tensor b (397) at non-singleton dimension 0

In [143]:
for (layer, param) in enumerate(model.parameters()):
    print("Layer {} , Parameters: {}".format(layer, param.shape))

Layer 0 , Parameters: torch.Size([399, 784])
Layer 1 , Parameters: torch.Size([399])
Layer 2 , Parameters: torch.Size([49, 399])
Layer 3 , Parameters: torch.Size([49])
Layer 4 , Parameters: torch.Size([10, 49])
Layer 5 , Parameters: torch.Size([10])


In [144]:
neurons_to_prune = []
for p in model.parameters():
        if len(p.data.size()) != 1:
            normed_weights = p.data.abs()
            l1_norm_layer = []
            for neuron_idx in range(normed_weights.shape[0]):
                l1_norm_layer.append(torch.sum(normed_weights[neuron_idx, :]).item())
            neurons_to_prune.append(torch.argmin(torch.FloatTensor(l1_norm_layer)))
            neurons_to_prune.append(torch.argmin(torch.FloatTensor(l1_norm_layer)))
                

In [145]:
list(model.parameters())

[Parameter containing:
 tensor([[ 3.1660e-02, -1.8102e-02,  4.2061e-03,  ...,  1.7766e-02,
          -3.1633e-04, -1.8067e-02],
         [-2.5592e-02,  2.2755e-02,  1.2970e-03,  ..., -1.2253e-02,
          -3.2247e-02,  2.5801e-02],
         [-9.1272e-03, -2.3506e-02,  8.0663e-03,  ..., -2.2597e-02,
           2.9777e-02,  1.8771e-03],
         ...,
         [ 5.9019e-04, -2.4208e-02,  3.1661e-02,  ..., -6.7178e-03,
          -7.4413e-03, -7.8642e-03],
         [ 4.3384e-05,  1.8712e-03,  9.2949e-03,  ...,  2.0801e-03,
          -2.5955e-02,  2.3680e-02],
         [ 1.5148e-02, -2.0020e-02,  3.1257e-02,  ..., -1.9081e-02,
          -1.4316e-02, -1.8939e-02]], requires_grad=True),
 Parameter containing:
 tensor([-0.0196, -0.0180,  0.0159,  0.0038, -0.0150,  0.0293,  0.0356,  0.0246,
         -0.0261, -0.0162,  0.0278,  0.0214, -0.0156, -0.0144, -0.0240, -0.0031,
         -0.0182,  0.0287, -0.0281, -0.0090,  0.0309, -0.0089, -0.0104, -0.0102,
          0.0375, -0.0281,  0.0305, -0.0198, 

In [169]:
param_list = list(model.parameters())
for i, neuron_idx in enumerate(neurons_to_prune):
    idx_weights = param_list[i]
    if i < len(param_list) - 2:
        y = torch.cat((idx_weights[0:neuron_idx], idx_weights[neuron_idx+1:]))
        if i > 1 and len(idx_weights.shape) > 1:

            y = torch.cat((y[:, 0:neuron_idx-1], y[:, neuron_idx:]), axis=1)
    elif i > 1 and len(idx_weights.shape) > 1:
        y = torch.cat((idx_weights[:, 0:neuron_idx-1], idx_weights[:, neuron_idx:]), axis=1)
    else:
        y = idx_weights
    idx_weights.data = y

In [170]:
for (layer, param) in enumerate(model.parameters()):
    print("Layer {} , Parameters: {}".format(layer, param.shape))

Layer 0 , Parameters: torch.Size([397, 784])
Layer 1 , Parameters: torch.Size([397])
Layer 2 , Parameters: torch.Size([47, 397])
Layer 3 , Parameters: torch.Size([47])
Layer 4 , Parameters: torch.Size([10, 47])
Layer 5 , Parameters: torch.Size([10])


In [171]:
model.state_dict()['input_layer.weight']

tensor([[ 3.1660e-02, -1.8102e-02,  4.2061e-03,  ...,  1.7766e-02,
         -3.1633e-04, -1.8067e-02],
        [-2.5592e-02,  2.2755e-02,  1.2970e-03,  ..., -1.2253e-02,
         -3.2247e-02,  2.5801e-02],
        [-9.1272e-03, -2.3506e-02,  8.0663e-03,  ..., -2.2597e-02,
          2.9777e-02,  1.8771e-03],
        ...,
        [ 5.9019e-04, -2.4208e-02,  3.1661e-02,  ..., -6.7178e-03,
         -7.4413e-03, -7.8642e-03],
        [ 4.3384e-05,  1.8712e-03,  9.2949e-03,  ...,  2.0801e-03,
         -2.5955e-02,  2.3680e-02],
        [ 1.5148e-02, -2.0020e-02,  3.1257e-02,  ..., -1.9081e-02,
         -1.4316e-02, -1.8939e-02]])

In [172]:
updated_model =  Network({
        "network":{
            'input_layer': {
                "units": 784,
                
                },
            'hidden_layer': [{
                    "units": 397, 
                    "type": "Linear"
                }, 
                {
                    "units": 47, 
                    "activation": "relu",
                    "type": "Linear"

                }],
            'output_layer': {
                "units": 10,
                "activation": "softmax",
                "type": "Linear"
                }
        }
    })

softmax


In [173]:
updated_model.load_state_dict(model.state_dict())

<All keys matched successfully>

In [174]:
criterion = nn.CrossEntropyLoss()
updated_optimizer = torch.optim.SGD(updated_model.parameters(), lr=learning_rate,
                      momentum=momentum)



In [175]:
updated_optimizer.load_state_dict(optimizer.state_dict())

In [176]:
optimizer = updated_optimizer

In [177]:
model = updated_model

In [178]:
model

Network(
  (input_layer): Linear(in_features=784, out_features=397, bias=True)
  (hidden_layers): ModuleList(
    (0): Linear(in_features=397, out_features=47, bias=True)
  )
  (output_layer): Linear(in_features=47, out_features=10, bias=True)
)